In [1]:
from __future__ import print_function, division
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [2]:
# 启动 Spark （如果你已经启动就不需要）
spark = SparkSession.builder.master("local[2]") \
   .appName("test") \
   .enableHiveSupport() \
   .getOrCreate()

sc = spark.sparkContext

## 讀取資料

In [53]:
!ls -lh ../data

total 196M
-rw-r--r--. 1 root root  188 Nov 10  2017 json_example.json
-rw-r--r--. 1 root root 4.5K Dec 27  2017 kmeans_data.txt
-rw-r--r--. 1 root root 196M Nov  5 16:28 NASA_access_log_Jul95
-rw-r--r--. 1 root root  11K Nov 10  2017 NASA_access_log_Jul95_100
-rw-r--r--. 1 root root  561 Nov 10  2017 ratings.csv
-rw-r--r--. 1 root root 103K Dec 27  2017 sample_libsvm_data.txt
-rw-r--r--. 1 root root 2.7K Nov 10  2017 shakespear.txt
-rw-r--r--. 1 root root  28K Dec 27  2017 titanic_test.csv
-rw-r--r--. 1 root root  60K Dec 27  2017 titanic_train.csv


In [3]:
df = spark.read.csv('../data/ratings.csv', header=True)

In [4]:
df.show()

+------+-------+------+----------+
|userid|movieid|rating|        ts|
+------+-------+------+----------+
|     3|   6539|     5|1133571238|
|     3|   7153|     4|1133571171|
|     3|   7155|   3.5|1164885564|
|     3|   8529|     4|1136075616|
|     3|   8533|   4.5|1136418593|
|     3|   8783|     5|1136075857|
|     3|  27821|   4.5|1136418616|
|     3|  33750|   3.5|1164885688|
|     3|  33750|   3.5|1164887688|
|     3|    344|  null| 844416742|
|     4|     21|     3| 844416980|
|     4|     34|     5| 844416936|
|     4|     39|     3| 844417037|
|     4|    110|     5| 844416866|
|     4|    150|     5| 844416656|
|     4|    153|     5| 844416699|
|     4|    161|     5| 844416835|
|     4|    165|     5| 844416699|
|     4|    208|     3| 844416866|
|     4|    231|     1| 844416742|
+------+-------+------+----------+
only showing top 20 rows



## 选择栏位

In [5]:
## select userID from data

In [6]:
df.select("userID").show()

+------+
|userID|
+------+
|     3|
|     3|
|     3|
|     3|
|     3|
|     3|
|     3|
|     3|
|     3|
|     3|
|     4|
|     4|
|     4|
|     4|
|     4|
|     4|
|     4|
|     4|
|     4|
|     4|
+------+
only showing top 20 rows



In [7]:
## select userID , movieID from data

In [8]:
df.select("userID", "movieID").show()

+------+-------+
|userID|movieID|
+------+-------+
|     3|   6539|
|     3|   7153|
|     3|   7155|
|     3|   8529|
|     3|   8533|
|     3|   8783|
|     3|  27821|
|     3|  33750|
|     3|  33750|
|     3|    344|
|     4|     21|
|     4|     34|
|     4|     39|
|     4|    110|
|     4|    150|
|     4|    153|
|     4|    161|
|     4|    165|
|     4|    208|
|     4|    231|
+------+-------+
only showing top 20 rows



## 操作栏位

In [9]:
## select userID as id from data

In [30]:
df.selectExpr("userID as id").show()  # 当用Expr的时候, 里面的语句跟sql是一样的

+---+
| id|
+---+
|  3|
|  3|
|  3|
|  3|
|  3|
|  3|
|  3|
|  3|
|  3|
|  3|
|  4|
|  4|
|  4|
|  4|
|  4|
|  4|
|  4|
|  4|
|  4|
|  4|
+---+
only showing top 20 rows



In [31]:
## select movieID, rating * 2 as rating_2

In [32]:
df.selectExpr("movieID", "rating * 2 as rating_2").show()

+-------+--------+
|movieID|rating_2|
+-------+--------+
|   6539|    10.0|
|   7153|     8.0|
|   7155|     7.0|
|   8529|     8.0|
|   8533|     9.0|
|   8783|    10.0|
|  27821|     9.0|
|  33750|     7.0|
|  33750|     7.0|
|    344|    null|
|     21|     6.0|
|     34|    10.0|
|     39|     6.0|
|    110|    10.0|
|    150|    10.0|
|    153|    10.0|
|    161|    10.0|
|    165|    10.0|
|    208|     6.0|
|    231|     2.0|
+-------+--------+
only showing top 20 rows



In [33]:
df.printSchema()

root
 |-- userid: string (nullable = true)
 |-- movieid: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- ts: string (nullable = true)



In [34]:
df.selectExpr("cast(rating as DOUBLE)").printSchema()  #转换类别

root
 |-- rating: double (nullable = true)



## 筛选

In [35]:
# select * from data where rating > 3

In [36]:
df.filter("rating > 3").show()

+------+-------+------+----------+
|userid|movieid|rating|        ts|
+------+-------+------+----------+
|     3|   6539|     5|1133571238|
|     3|   7153|     4|1133571171|
|     3|   8529|     4|1136075616|
|     3|   8533|   4.5|1136418593|
|     3|   8783|     5|1136075857|
|     3|  27821|   4.5|1136418616|
|     4|     34|     5| 844416936|
|     4|    110|     5| 844416866|
|     4|    150|     5| 844416656|
|     4|    153|     5| 844416699|
|     4|    161|     5| 844416835|
|     4|    165|     5| 844416699|
|     4|    266|     5| 844417070|
|     4|    316|     5| 844416742|
|     4|    317|     5| 844417037|
|     4|    329|     5| 844416796|
+------+-------+------+----------+



In [22]:
# select * from data where userID = 3 and rating >3

In [46]:
df.filter("userid = 3 and rating > 3").show()

+------+-------+------+----------+
|userid|movieid|rating|        ts|
+------+-------+------+----------+
|     3|   6539|     5|1133571238|
|     3|   7153|     4|1133571171|
|     3|   8529|     4|1136075616|
|     3|   8533|   4.5|1136418593|
|     3|   8783|     5|1136075857|
|     3|  27821|   4.5|1136418616|
+------+-------+------+----------+



In [25]:
# select userID, rating from data where userId = 3 and rating > 3

In [47]:
df.filter("userID = 3 and rating > 3").select("userID", "rating").show()

+------+------+
|userID|rating|
+------+------+
|     3|     5|
|     3|     4|
|     3|     4|
|     3|   4.5|
|     3|     5|
|     3|   4.5|
+------+------+



In [48]:
df.select("userID", "rating").filter("userID = 3 and rating > 3").show()

+------+------+
|userID|rating|
+------+------+
|     3|     5|
|     3|     4|
|     3|     4|
|     3|   4.5|
|     3|     5|
|     3|   4.5|
+------+------+



In [63]:
df.filter("userid != 3 ").show()

+------+-------+------+---------+
|userid|movieid|rating|       ts|
+------+-------+------+---------+
|     4|     21|     3|844416980|
|     4|     34|     5|844416936|
|     4|     39|     3|844417037|
|     4|    110|     5|844416866|
|     4|    150|     5|844416656|
|     4|    153|     5|844416699|
|     4|    161|     5|844416835|
|     4|    165|     5|844416699|
|     4|    208|     3|844416866|
|     4|    231|     1|844416742|
|     4|    253|     3|844416834|
|     4|    266|     5|844417070|
|     4|    292|     3|844416796|
|     4|    316|     5|844416742|
|     4|    317|     5|844417037|
|     4|    329|     5|844416796|
|     4|    344|     2|844416699|
|     4|    349|     3|844416699|
|     4|   null|  null|     null|
+------+-------+------+---------+



## 聚合运算

In [49]:
# select count(*) from data

In [50]:
df.count()

29

In [51]:
df.agg({'userID': 'count'}).show()

+-------------+
|count(userID)|
+-------------+
|           29|
+-------------+



In [54]:
# select count(*) from data where userID =1

In [56]:
df.filter("userID = 3").count()

10

In [57]:
# select count(userID) from data, select avg(rating) from data 

In [58]:
df.agg({'userID': 'count', 'rating': 'avg'}).show()

+-----------------+-------------+
|      avg(rating)|count(userID)|
+-----------------+-------------+
|4.018518518518518|           29|
+-----------------+-------------+



## Group By

In [59]:
# 计算每个user 评比了多少部电影，平均分数为何?

select userID, count(*), avg(rating)
from data
group userID

In [64]:
df.groupBy('userID').agg({'movieID': 'count', 'rating':'avg'}).show()  #要先groupby才能做aggregation

+------+------------------+--------------+
|userID|       avg(rating)|count(movieID)|
+------+------------------+--------------+
|     3| 4.166666666666667|            10|
|     4|3.9444444444444446|            18|
+------+------------------+--------------+



In [65]:
from pyspark.sql.functions import *
df.groupBy('userID').agg(count('movieID'), round(avg(df.rating),2)).show()

+------+--------------+---------------------+
|userID|count(movieID)|round(avg(rating), 2)|
+------+--------------+---------------------+
|     3|            10|                 4.17|
|     4|            18|                 3.94|
+------+--------------+---------------------+



## Join tables

![sql](http://4.bp.blogspot.com/-_HsHikmChBI/VmQGJjLKgyI/AAAAAAAAEPw/JaLnV0bsbEo/s1600/sql%2Bjoins%2Bguide%2Band%2Bsyntax.jpg)

In [66]:
d = [{'name': 'Alice', 'age': 1}, {'name': 'Bryan', 'age': 3}, {'name': 'Cool', 'age':2}]
df_profile = spark.createDataFrame(d)
df_profile.show()

/usr/local/spark/python/pyspark/sql/session.py:340: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+---+-----+
|age| name|
+---+-----+
|  1|Alice|
|  3|Bryan|
|  2| Cool|
+---+-----+



In [67]:
d = [{'name': 'Jason', 'child': 'Alice'}, 
     {'name': 'Bill', 'child': 'Bryan'}, 
     {'name': 'Sera', 'child': 'Bryan'}, 
     {'name': 'Jill', 'child': 'Ken'}]
df_parents = spark.createDataFrame(d)
df_parents.show()

+-----+-----+
|child| name|
+-----+-----+
|Alice|Jason|
|Bryan| Bill|
|Bryan| Sera|
|  Ken| Jill|
+-----+-----+



In [61]:
df_profile.join(df_parents, df_profile.name == df_parents.child).show() #inner join

+---+-----+-----+-----+
|age| name|child| name|
+---+-----+-----+-----+
|  1|Alice|Alice|Jason|
|  3|Bryan|Bryan| Bill|
|  3|Bryan|Bryan| Sera|
+---+-----+-----+-----+



In [68]:
df_profile.join(df_parents, df_profile.name == df_parents.child, 'left').show()  # leftjoin

+---+-----+-----+-----+
|age| name|child| name|
+---+-----+-----+-----+
|  2| Cool| null| null|
|  1|Alice|Alice|Jason|
|  3|Bryan|Bryan| Bill|
|  3|Bryan|Bryan| Sera|
+---+-----+-----+-----+



In [69]:
df_profile.join(df_parents, df_profile.name == df_parents.child, 'right').show()  #rightjoin

+----+-----+-----+-----+
| age| name|child| name|
+----+-----+-----+-----+
|   1|Alice|Alice|Jason|
|null| null|  Ken| Jill|
|   3|Bryan|Bryan| Bill|
|   3|Bryan|Bryan| Sera|
+----+-----+-----+-----+



In [70]:
df_profile.join(df_parents, df_profile.name == df_parents.child, 'outer').show()  #outerjoin

+----+-----+-----+-----+
| age| name|child| name|
+----+-----+-----+-----+
|   2| Cool| null| null|
|   1|Alice|Alice|Jason|
|null| null|  Ken| Jill|
|   3|Bryan|Bryan| Bill|
|   3|Bryan|Bryan| Sera|
+----+-----+-----+-----+



## 注册为 SQL 表

In [71]:
df.registerTempTable("table")

In [72]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |    table|       true|
+--------+---------+-----------+



In [73]:
spark.sql("select * from table limit 10").show()

+------+-------+------+----------+
|userid|movieid|rating|        ts|
+------+-------+------+----------+
|     3|   6539|     5|1133571238|
|     3|   7153|     4|1133571171|
|     3|   7155|   3.5|1164885564|
|     3|   8529|     4|1136075616|
|     3|   8533|   4.5|1136418593|
|     3|   8783|     5|1136075857|
|     3|  27821|   4.5|1136418616|
|     3|  33750|   3.5|1164885688|
|     3|  33750|   3.5|1164887688|
|     3|    344|  null| 844416742|
+------+-------+------+----------+



In [74]:
def run_sql(sql_string):
    spark.sql(sql_string).show()

In [75]:
sql_string = """
select * 
from table
"""
run_sql(sql_string)

+------+-------+------+----------+
|userid|movieid|rating|        ts|
+------+-------+------+----------+
|     3|   6539|     5|1133571238|
|     3|   7153|     4|1133571171|
|     3|   7155|   3.5|1164885564|
|     3|   8529|     4|1136075616|
|     3|   8533|   4.5|1136418593|
|     3|   8783|     5|1136075857|
|     3|  27821|   4.5|1136418616|
|     3|  33750|   3.5|1164885688|
|     3|  33750|   3.5|1164887688|
|     3|    344|  null| 844416742|
|     4|     21|     3| 844416980|
|     4|     34|     5| 844416936|
|     4|     39|     3| 844417037|
|     4|    110|     5| 844416866|
|     4|    150|     5| 844416656|
|     4|    153|     5| 844416699|
|     4|    161|     5| 844416835|
|     4|    165|     5| 844416699|
|     4|    208|     3| 844416866|
|     4|    231|     1| 844416742|
+------+-------+------+----------+
only showing top 20 rows



In [76]:
sql_string = """
select userid as id 
from table
"""
run_sql(sql_string)

+---+
| id|
+---+
|  3|
|  3|
|  3|
|  3|
|  3|
|  3|
|  3|
|  3|
|  3|
|  3|
|  4|
|  4|
|  4|
|  4|
|  4|
|  4|
|  4|
|  4|
|  4|
|  4|
+---+
only showing top 20 rows



In [77]:
sql_string = """
select userID, count(*)
from table
group by userID
"""
run_sql(sql_string)

+------+--------+
|userID|count(1)|
+------+--------+
|     3|      10|
|     4|      19|
+------+--------+

